In [8]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [9]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # implement the forward pass
        t = t

        # first hidden layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # second hidden layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # 3 hidden layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # output layer
        t = self.out(t)

        return t




In [29]:
from torch.optim import Adam
from torch.utils.data import DataLoader

data_loader = DataLoader(train_set, batch_size=10)

network = Network()
print(network.conv1.weight.shape)

for param in network.parameters():
    print(param.shape)


train_loader = DataLoader(train_set, batch_size=100)
batch = next(iter(train_loader)) # Getting a batch
images, labels = batch

torch.Size([6, 1, 5, 5])
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([12, 6, 5, 5])
torch.Size([12])
torch.Size([120, 192])
torch.Size([120])
torch.Size([60, 120])
torch.Size([60])
torch.Size([10, 60])
torch.Size([10])


## Train Using A Single Batch

In [46]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = Adam(network.parameters(), lr=0.01)

batch = next(iter(train_loader)) # Get Batch
images, labels = batch

preds = network(images) # Pass Batch
loss = F.cross_entropy(preds, labels) # Calculate Loss

loss.backward() # Calculate Gradients
optimizer.step() # Update Weights

print(f'loss 1: {loss.item()}, Number correct in 100: {get_num_correct(preds, labels)}')
print("\nPerfrormance after one update:")
preds = network(images)
loss = F.cross_entropy(preds, labels)
print(f'loss 2: {loss.item()}, Number correct in 100: {get_num_correct(preds, labels)}')


loss 1: 2.301861524581909, Number correct in 100: 8

Perfrormance after one update:
loss 2: 2.2683188915252686, Number correct in 100: 15


## Training Loop

In [49]:
## Training With All Batches (Single Epoch)


In [ ]:
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [54]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = Adam(network.parameters(), lr=0.01)



for num in range(3):
    total_loss = 0
    total_correct = 0
    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch:", num, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

epoch: 0 total_correct: 47979 loss: 322.62883473932743
epoch: 1 total_correct: 51493 loss: 228.04275259375572
epoch: 2 total_correct: 52215 loss: 209.34832498431206


In [52]:
total_correct/len(train_set)*100

78.10666666666667

## Tensoboard

In [61]:
import torch
torch.__version__

'1.5.0'

In [57]:
from torch.utils.tensorboard import SummaryWriter

In [73]:
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = Adam(network.parameters(), lr=0.01)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):
    
    total_loss = 0
    total_correct = 0
    
    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram(
        'conv1.weight.grad'
        ,network.conv1.weight.grad
        ,epoch
    )
    
    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )
    
tb.close()

epoch 0 total_correct: 46140 loss: 363.12742882966995
epoch 1 total_correct: 51184 loss: 236.89719860255718
epoch 2 total_correct: 51877 loss: 218.85394302010536
epoch 3 total_correct: 52279 loss: 209.67606389522552
epoch 4 total_correct: 52385 loss: 206.2150332480669
epoch 5 total_correct: 52519 loss: 203.7906699627638
epoch 6 total_correct: 52646 loss: 199.3007475733757
epoch 7 total_correct: 52752 loss: 198.54001632332802
epoch 8 total_correct: 52852 loss: 194.47149212658405
epoch 9 total_correct: 52986 loss: 191.62672293186188


In [70]:
!tensorboard --logdir=runs

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.2.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


## Hyperparameter Tuning

In [87]:
from itertools import product

batch_size = 100
lr = 0.01

network = Network()
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size
)
optimizer = Adam(
    network.parameters(), lr=lr
)

In [88]:
tb = SummaryWriter(comment=f' batch_size={batch_size} lr={lr}')


In [89]:
batch_size_list = [100, 1000, 10000]
lr_list = [.01, .001, .0001, .00001]

In [90]:
parameters = dict(
    lr = [.01, .001]
    ,batch_size = [100, 1000]
    ,shuffle = [True, False]
)

In [91]:
param_values = [v for v in parameters.values()]
for lr, batch_size, shuffle in product(*param_values): 
    print (lr, batch_size, shuffle)

0.01 100 True
0.01 100 False
0.01 1000 True
0.01 1000 False
0.001 100 True
0.001 100 False
0.001 1000 True
0.001 1000 False


In [96]:
for epoch in range(2):
    total_loss = 0
    total_correct = 0
    for lr, batch_size, shuffle in product(*param_values): 
        comment = f' batch_size={batch_size} lr={lr} shuffle={shuffle}'

        train_loader = torch.utils.data.DataLoader(
            train_set
            ,batch_size=batch_size
            ,shuffle=shuffle 
        )

        optimizer = Adam(
            network.parameters(), lr=lr
        )
        images, labels = next(iter(train_loader))
        grid = torchvision.utils.make_grid(images)

        comment=f' batch_size={batch_size} lr={lr}'
        tb = SummaryWriter(comment=comment)
        tb.add_image('images', grid)
        tb.add_graph(network, images)
        
        
        images, labels = batch # Get Batch
        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss
        optimizer.zero_grad() # Zero Gradients
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item() * batch_size
        total_correct += get_num_correct(preds, labels)

    tb.add_scalar(
        'Loss', total_loss, epoch
    )
    tb.add_scalar(
        'Number Correct', total_correct, epoch
    )
    tb.add_scalar(
        'Accuracy', total_correct / len(train_set), epoch
    )

    for name, param in network.named_parameters():
        tb.add_histogram(name, param, epoch)
        tb.add_histogram(f'{name}.grad', param.grad, epoch)

    print(
        "epoch", epoch
        ,"total_correct:", total_correct
        ,"loss:", total_loss
    )  
tb.close()

epoch 0 total_correct: 781 loss: 247.43487052619457
epoch 1 total_correct: 798 loss: 57.53480987623334


## Class Run Builder:


In [116]:
from collections import namedtuple, OrderedDict
class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

In [117]:
params = OrderedDict(
    lr = [.01, .001]
    ,batch_size = [1000, 10000]
)

In [120]:
runs = RunBuilder.get_runs(params)

In [121]:
runs

[Run(lr=0.01, batch_size=1000),
 Run(lr=0.01, batch_size=10000),
 Run(lr=0.001, batch_size=1000),
 Run(lr=0.001, batch_size=10000)]